In [5]:
import os
import flow
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', style='darkgrid')
from IPython.display import clear_output
import pandas as pd

In [4]:
mouse = 'OA27'
date_tags = None
all_dates = flow.metadata.metadata.dates(mouse, tags=date_tags)
ids = {}
for date in all_dates:
    # get your cells ids per day and turn them into ints
    # save these into dict where keys are days and values are ids
    date_ids = flow.xday._read_crossday_ids(mouse, date)
    date_ids = [int(s) for s in date_ids]
    ids[date] = date_ids

# get the maximum cell number
max_id_list = [np.max(ids[s]) for s in ids]
max_id = np.max(max_id_list)
min_id_list = [np.min(ids[s]) for s in ids]
min_id = np.min(min_id_list)
print('min_id: ' + str(min_id) + ', max_id: ' + str(max_id))


min_id: 1, max_id: 1555


In [6]:
mouse = 'CC175'
date_tags = None
all_dates = flow.metadata.metadata.dates(mouse, tags=date_tags)
print(all_dates)
run_tags = ['kelly', 'hungry']

# loop through dates and build a dict where keys are dates
# and values are np arrays tensor cellsxtimesxtrials
plus_traces = {}
plus_ids = {}
for date in all_dates:
    all_runs = flow.metadata.metadata.runs(mouse, date, tags=run_tags)
    
    # get your cells ids per day and turn them into ints
    # save these into dict where keys are days and values are ids
    date_ids = flow.xday._read_crossday_ids(mouse, date)
    date_ids = [int(s) for s in date_ids]
    plus_ids[date] = date_ids
    
    count = 1
    run_traces = np.array([])
    for run in all_runs:
        t2p = flow.paths.gett2p(mouse, date, run)
        print(t2p.cses())
#         print(t2p)
        
        # get plus traces checking for possible t2p.cses() 
        single_run_traces = t2p.cstraces('', start_s=-1, end_s=6, trace_type='dff',
                                         cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                                        baseline_to_stimulus=True)
        
        # concatenate runs for a given day
        if count == 1:
            run_traces = single_run_traces
            count = count + 1
        else:
            run_traces = np.concatenate((run_traces, single_run_traces), axis=2)
                
#         print(np.shape(single_run_traces))
#         print(np.shape(run_traces))
        
    # build a dict where you have concatenated each run for a given day   
    plus_traces[date] = run_traces
#     print(traces.keys())


[170116, 170118, 170119, 170125, 170127, 170128, 170129, 170131, 170201, 170202, 170204, 170206, 170207, 170208, 170209, 170210, 170211, 170214, 170215, 170216, 170217, 170220, 170223, 170224, 170303, 170308, 170318, 170320, 170321, 170322, 170323, 170326, 170328, 170403, 170404]
['neutral', 'plus', 'minus', 'blank', 'reward', 'punishment']
['neutral', 'plus', 'minus', 'blank', 'reward', 'punishment']
['neutral', 'plus', 'minus', 'blank', 'reward', 'punishment']
['neutral', 'plus', 'minus', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 'reward', 'punishment']
['plus', 'pavlovian', 'blank', 

In [8]:
# pull all trials for individual cells
# build a dict where each value is a cell id
cells = {}
for cell_id in range(min_id,max_id+1): # add 1 because py indexing is not inclusive 
    
#     print('Cell number: ' + str(cell_id))
    start_flag = False
    cell_traces = np.array([])

    for date in plus_traces:
        
        # get the traces for one cell using unique id
        # get the index for a cell id on that day
        cell_ind = np.where(np.array(plus_ids[date])==cell_id)[0]
        
        # if cell is found on that day concatenate with existing traces
        if len(cell_ind) and start_flag == False:
            cell_traces = np.squeeze(plus_traces[date][cell_ind,:,:])
            start_flag = True
        elif len(cell_ind) and start_flag == True:
            cell_traces_today = np.squeeze(plus_traces[date][cell_ind,:,:])
            cell_traces = np.concatenate((cell_traces, cell_traces_today), axis=1)
#             print(np.shape(cell_traces))
            
    if len(cell_traces) == 0:
        print('Cell number: ' + str(cell_id) + ', no trials!')
            
    cells[cell_id] = cell_traces
    

In [ ]:
# loop through and plot your individual cells 
cmap = sns.diverging_palette(220, 10, as_cmap=True)
for cell_ids in cells:
    fig, ax = plt.subplots()
    sns.heatmap(cells[cell_ids].T, cmap=cmap, center=0, vmin=-1, vmax=1)
    ax.set_title('Cell #' + str(cell_ids))
#     print(np.shape(cells[cell_ids]))

#     plt.savefig(analysis_subfolder + '/nameX'+ '.png')
#     plt.show()
        
#     clear_output()
        
# TODO:
# 1. consider reformatiing into pandas dataframe to make seaborn plotting easier
# 2. you need to get a way to plot according to orientation of grating and not just CS, could add this to dataframe 

/Users/kelly_mcguire/anaconda3/envs/py27/lib/python2.7/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


KeyboardInterrupt: 

In [30]:
mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])
for run in runs:
    # get your t2p object
    t2p = run.trace2p()
    start_time = -1
    end_time = 6
    trace_type = 'dff'
    
    cell_ids = flow.xday._read_crossday_ids(run.mouse, run.date)
    cell_ids = [int(s) for s in cell_ids]
    
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                baseline_to_stimulus=True)
    
    print(str(run) + '  traces: ' + str(np.shape(run_traces)[0]) + ', ids: ' + str(len(cell_ids)))

OA27_170116_2  traces: 264, ids: 264
OA27_170116_3  traces: 264, ids: 264
OA27_170118_2  traces: 264, ids: 253
OA27_170118_3  traces: 264, ids: 253
OA27_170119_1  traces: 264, ids: 237
OA27_170119_2  traces: 264, ids: 237


KeyboardInterrupt: 

In [37]:
print(len(cell_ids))
print(range(np.shape(run_traces)[0])[-1])
print(range(np.shape(run_traces)[0])[0])
print(np.shape(run_traces))

237
263
0
(264, 217, 152)


In [3]:
# tag_options = ['hungry', 'sated', 'disengaged', 'contrast', 'orientation_mapping', 'retinotopy']
# TODO add a seconds dataframe which is the trial variavles  

mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

# triggering parameters
start_time = -1
end_time = 6
trace_type = 'dff'

trial_list = []
chunk = 1
for run in runs:

    # get your t2p object
    t2p = run.trace2p()

    # get your cell# from xday alignment
    # use to index along axis=0 in cstraces/run_traces
    cell_ids = flow.xday._read_crossday_ids(run.mouse, run.date)
    cell_ids = [int(s) for s in cell_ids]
    max_id = np.max(cell_ids)
    # print(np.shape(cell_ids))

    # get all of your trial variables of interest
    # use to index along axis=2 in cstraces/run_traces
    oris = []
    css = []
    for trial in t2p.d['condition']:
        codename = t2p.d['codes'].keys()[t2p.d['codes'].values().index(t2p.d['condition'][trial])]
        oriname = t2p.d['orientations'][codename]
        css.append(codename)
        oris.append(oriname)

    oris = np.array(oris)
    # print(np.shape(oris))
    # print(oris)
    # print(css)

    # trigger all trials around stimulus onsets
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                    cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                    baseline_to_stimulus=True)

    # make timestamps 
    timestep = 1/np.round(t2p.d['framerate'])
    timestamps = np.concatenate((np.arange(start_time, 0, timestep), 
                                 np.arange(0, end_time, timestep)))

    # loop through and append each trial (slice of cstraces)
#     print(len([int(trial)] * np.shape(run_traces)[1]))
    for trial in range(np.shape(run_traces)[2]):
        for cell in range(np.shape(run_traces)[0]):
            # to take car of an indexing error from origianl pull don't look beyond known ids
            if cell >= len(cell_ids):
                continue
            index = pd.MultiIndex.from_arrays([
                        [run.mouse] * np.shape(run_traces)[1], 
                        [run.date] * np.shape(run_traces)[1],
                        [run.run] * np.shape(run_traces)[1],
                        [int(trial)] * np.shape(run_traces)[1],
                        [cell_ids[cell]] * np.shape(run_traces)[1],
                        timestamps
                        ],
                        names=['mouse', 'date', 'run', 'trial_idx', 'cell_idx', 'timestamp'])

            # append all trials across all runs together in a list
            trial_list.append(pd.DataFrame({'trace': np.squeeze(run_traces[cell,:,trial])}, index=index))

    # save data in million-trial chunks
    if len(trial_list) > 1000000:

        # concatenate and save current db
        trial_df = pd.concat(trial_list, axis=0)
        save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_klg_' + trace_type + '_' + str(chunk) +'.pkl')
        trial_df.to_pickle(save_path)

        # reset trial_list
        trial_list = []

        # increase the multiplier
        chunk = chunk + 1

    # clear your t2p
    run._t2p = None
    print('Run: ' + str(run) + ': ' + str(len(trial_list)))
# concatenate all runs together in final dataframe 
# trial_df = pd.concat(trial_list, axis=0)

# save
# save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_klg_' + trace_type + '.pkl')
# trial_df.to_pickle(save_path)

# concatenate and save final db
trial_df = pd.concat(trial_list, axis=0)
save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_klg_' + trace_type + '_' + str(chunk) +'.pkl')
trial_df.to_pickle(save_path)

Run: OA27_170116_2: 58080
Run: OA27_170116_3: 99264
Run: OA27_170118_2: 154924
Run: OA27_170118_3: 194392
Run: OA27_170119_1: 231127
Run: OA27_170119_2: 267151
Run: OA27_170125_2: 325937
Run: OA27_170125_3: 367167
Run: OA27_170125_4: 396959
Run: OA27_170127_2: 434314
Run: OA27_170127_3: 471669
Run: OA27_170128_2: 495229
Run: OA27_170128_3: 518789
Run: OA27_170128_4: 539917
Run: OA27_170129_2: 571669
Run: OA27_170129_3: 603421
Run: OA27_170129_4: 633877
Run: OA27_170129_5: 664765
Run: OA27_170131_4: 693449
Run: OA27_170131_5: 721931
Run: OA27_170131_6: 750413
Run: OA27_170201_3: 784811
Run: OA27_170201_4: 816869
Run: OA27_170201_5: 848927
Run: OA27_170201_6: 880517
Run: OA27_170201_7: 911171
Run: OA27_170202_2: 944671
Run: OA27_170202_3: 977921
Run: OA27_170202_4: 0
Run: OA27_170202_5: 31750
Run: OA27_170204_2: 61637
Run: OA27_170204_4: 91315
Run: OA27_170204_5: 118903
Run: OA27_170204_6: 145864
Run: OA27_170206_3: 179386
Run: OA27_170206_4: 211132
Run: OA27_170206_5: 245542
Run: OA27_1

In [ ]:
sns.heatmap(trial_df[trial_df.])

In [118]:
# tag_options = ['hungry', 'sated', 'disengaged', 'contrast', 'orientation_mapping', 'retinotopy']
# TODO add a seconds dataframe which is the trial variavles  

mouse = 'OA27'
runs = flow.metadata.RunSorter.frommeta(mice=[mouse])

# triggering parameters
start_time = -1
end_time = 6
trace_type = 'dff'

trial_list = []
chunk = 1
for run in runs:

    # get your t2p object
    t2p = run.trace2p()

    # get your cell# from xday alignment
    # use to index along axis=0 in cstraces/run_traces
    cell_ids = flow.xday._read_crossday_ids(run.mouse, run.date)
    cell_ids = [int(s) for s in cell_ids]
    max_id = np.max(cell_ids)
    # print(np.shape(cell_ids))

    # get all of your trial variables of interest
    # use to index along axis=2 in cstraces/run_traces
    oris = []
    css = []
    trialerror = []
    for trial in t2p.d['condition']:

        # get cs and ori
        codename = t2p.d['codes'].keys()[t2p.d['codes'].values().index(t2p.d['condition'][trial])]
        oriname = t2p.d['orientations'][codename]
        css.append(codename)
        oris.append(oriname)

        # get trialerror
        trialerror.append(t2p.d['trialerror'][trial])

    oris = np.array(oris)
    # print(np.shape(oris))
    # print(oris)
    # print(css)

    # trigger all trials around stimulus onsets
    run_traces = t2p.cstraces('', start_s=start_time, end_s=end_time, trace_type=trace_type,
                    cutoff_before_lick_ms=-1, errortrials=-1, baseline=None,
                    baseline_to_stimulus=True)

    # loop through and append each trial (slice of cstraces)
    for trial in range(np.shape(run_traces)[2]):
        index = pd.MultiIndex.from_arrays([
                    [run.mouse], 
                    [run.date],
                    [run.run],
                    [oris[trial]],
                    [css[trial]],
                    []
                    ],
                    names=['mouse', 'date', 'run', 'orientation', 'condition', 'trialerror'])

        # append all trials across all runs together in a list
        trial_list.append(pd.DataFrame({'trial_idx': int(range(np.shape(run_traces)[2]))}, index=index))

    # clear your t2p to save RAM
    run._t2p = None
    print('Run: ' + str(run) + ': ' + str(len(trial_list)))

# concatenate all runs together in final dataframe 
trial_df = pd.concat(trial_list, axis=0)

# save
save_path = os.path.join(flow.paths.outd, str(runs[0].mouse) + '_df_klg_' + trace_type + 'trial_meta.pkl')
trial_df.to_pickle(save_path)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12603360 entries, (OA27, 170116, 2, 0, 1, -1.0) to (OA27, 170116, 2, 219, 253, 5.967741935483871)
Data columns (total 1 columns):
trace    float64
dtypes: float64(1)
memory usage: 204.3+ MB


In [100]:
start_flag = False
cell_ind = 1
for date in plus_traces:

    # get the traces for one cell using unique id
    # get the index for a cell id on that day
    cell_ind = np.where(np.array(plus_ids[date])==cell_id)[0]
    print(cell_ind)
    if len(cell_ind):
        print('exists')
        

[0]
exists
[22]
exists
[7]
exists
[104]
exists
[22]
exists
[]
[]
[]
[]
[35]
exists
[]
[]
[]
[]
[]
[]
[]
[156]
exists
[]
[199]
exists
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
